# Загрузка библиотек
Загрузим необходимые библиотеки.

In [51]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

# Определение функций
У каждой квартиры 15 параметров.
Определим функции для извлечения этих параметров (и вспомогательные функции).

.) Вспомогательная функция №1:

In [52]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

1) Для извлечения Price – цены:

In [53]:
def getPrice(flat_page):
    try:
        price = flat_page.find('div', attrs={'class':'object_descr_price'})
        price = re.split('<div>|руб|\W', str(price))
        price = "".join([i for i in price if i.isdigit()][-4:])
        return int(price)
    except: 
        return np.NaN
#P.S.: В вашем коде была одна другая строчка:
#        price = "".join([i for i in price if i.isdigit()][-3:])
#      Но на сайте есть квартиры стоимостью больше миллиарда.
#      => Нужно писать:
#        price = "".join([i for i in price if i.isdigit()][-4:]) ,
#      т.е. брать 4 элемента из списка с ценой.
#      Для квартир стоимостью меньше миллиарда код будет работать по-прежнему.

2) Для извлечения Dist – расстояния от центра в км.:

In [54]:
#Координаты центра Москвы согласно Яндекс: (37.609218,55.753559)
#Расстояние вычисляется по теореме Пифагора
lat_center = 55.753559
lon_center = 37.609218
def getDist(flat_page):
    try:
        coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
        coords = re.split('&amp|center=|%2C', str(coords))
        coords_list = []
        for item in coords:
            if item[0].isdigit():
                coords_list.append(item)
        lat = float(coords_list[0])
        lon = float(coords_list[1])
        return ((((lat_center - lat)**2)+((lon_center - lon)**2))**(1/2))/0.014723278711700902
    except: 
        return np.NaN

3) Для извлечения Rooms – числа комнат:

In [66]:
def getRoom(flat_page):
    try:
        rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
        rooms = html_stripper(rooms)
        room_number = ''
        for i in re.split('-|\n', rooms):
            if 'комн' in i:
                break
            else:
                room_number += i
        room_number = "".join(room_number.split())
        return room_number
    except: 
        return np.NaN

На странице выбранной квартиры находится таблица с ее описанием. Почти все остальные параметры можно взять из нее почти тем же образом, которым мы получили предыдущие параметры.
=> Продолжим извлекать параметры* (*для удобства я сделал это в том порядке, в котором они даны в описании к заданию).

4) Для извлечения New – типа дома (1 – первичный рынок, 0 - вторичка):

In [67]:
def getNew(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        new = re.split('Тип дома:|Тип продажи', table)[1]
        for i in re.split('-|\n', new):
            if 'вторичка' in i:
                return 0
            if 'новостройка' in i:
                return 1
        return np.NaN #Если там нет ни "вторичка", ни "новостройка"
    except: #Если там нет таблицы => нет информации
        return np.NaN

5) Для извлечения Totsp – общей площади квартиры, кв. м.:

In [68]:
def getTotsp(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        totsp = re.split('Общая площадь:|Площадь комнат', table)[1]
        for i in re.split('-|\n', totsp):
            if '\xa0м2' in i:
                t = re.split('\xa0', i)[0]
                return int(re.split('-|,',t)[0])
        return np.NaN
    except:
        return np.NaN

6) Для извлечения Livesp – жилой площади квартиры, кв. м.

In [69]:
def getLivesp(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        livesp = re.split('Жилая площадь:|Площадь кухни', table)[1]
        for i in re.split('-|\n', livesp):
            if '\xa0м2' in i:
                l = re.split('\xa0', i)[0]
                return int(re.split('-|,',l)[0])
        return np.NaN
    except: 
        return np.NaN

7) Для извлечения Kitsp – площади кухни, кв. м.:

In [70]:
def getKitsp(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        kitsp = re.split('Площадь кухни:|санузлов', table)[1]
        for i in re.split('-|\n', kitsp):
            if '\xa0м2' in i:
                k = re.split('\xa0', i)[0]
                return int(re.split('-|,',k)[0])
        return np.NaN
    except: 
        return np.NaN

8) Для извлечения Metrdist – расстояния до метро в минутах

In [71]:
def getMetrdist(flat_page):  
    try:
        metro = flat_page.find('span', attrs = {'class':'object_item_metro_comment'})
        metro = html_stripper(metro)
        metro = re.split('-|\n', metro)
        if metro[1][len(metro[1])-2].isdigit() == True:
                return int(metro[1][len(metro[1])-2] + metro[1][len(metro[1])-1])
        else:
            return int(metro[1][len(metro[1])-1])
    except: 
        return np.NaN

9) Для извлечения Walk – (адекватной) возможности дойти пешком от метро (1 – если пешком от метро, 0 – если на транспорте)

In [72]:
def getWalk(flat_page):
    try:
        metro = flat_page.find('span', attrs = {'class':'object_item_metro_comment'})
        metro = html_stripper(metro)
        metro = re.split('-|\n', metro)    
        for i in metro:
            if 'пешком' in i:
                return 1
        return 0
    except: 
        return np.NaN

10) Для извлечения Brick – типа дома (1 – кирпичный/монолит/жб, 0 – другой)

In [73]:
def getBrick(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        brick = re.split('Тип дома:|Тип продажи', table)[1]
        for i in re.split('-|\n', brick):
            i = i.replace(' ','')
            if ('кирпичный' in i) or ('монолитный' in i) or ('панельный' in i):
                return 1
        return 0
    except: 
        return np.NaN

11) Для извлечения Tel – наличия телефона (1 – если есть, 0 – нет):

In [74]:
def getTel(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        tel = re.split('Телефон:|Вид из окна', table)[1]
        for i in re.split('-|\n', tel):
            if 'да' in i:
                return 1
        return 0
    except: 
        return np.NaN

12) Для извлечения Bal – наличия балкона/лоджии (1 – есть балкон/лоджия, 0 – нет):

In [75]:
def getBal(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        bal = re.split('Балкон:|Лифт', table)[1]
        for i in re.split('-|\n', bal):
            if 'да' in i:
                return 1
        return 0
    except: 
        return np.NaN

13) Для извлечения Floor – номера этажа, на котором расположена квартира:

In [76]:
def getFloor(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        floor = re.split('Этаж:|Тип дома', table)[1]
        for i in re.split('-|\n', floor):
            if '\xa0' in i:
                f = i.replace(' ','')
                f = f.split('/',1)[0]
                return int(f.replace('\xa0',''))
        return np.NaN
    except: 
        return np.NaN

14) Для извлечения Nfloors – количества всех этажей в доме:

In [77]:
def getNfloors(flat_page):
    try:
        table = flat_page.find('table', attrs = {'class':'object_descr_props'})
        table = html_stripper(table)
        Nfloors = re.split('Этаж:|Тип дома', table)[1]
        for i in re.split('-|\n', Nfloors):
            if '\xa0' in i:
                Nf = i.replace(' ','')
                Nf = Nf.split('/',1)[1]
                return int(Nf.replace('\xa0',''))
        return np.NaN
    except: 
        return np.NaN

.) Вспомогательная функция №2:

In [78]:
#Нам также понадобится вспомогательная функция для удаления повторяющихся квартир (т.е. повторяющихся сылок) в датасете.
def delete_dublicates(links):
    oldlinks = set()
    oldlinks_add = oldlinks.add
    return [x for x in links if not (x in oldlinks or oldlinks_add(x))]

# Непосредственно сам процесс
Я постараюсь подойти к задаче гибко.

In [79]:
#Выкачивать весь ЦИАН - долго, но это можно сделать с помощью, так сказать, мастер ссылки:
Super_Link = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D={}&engine_version=2&offer_type=flat&p={}&room{}=1'
#С ее помощью можно задавать различные параметры поиска: район квартиры, количество комнат в ней и номер страницы из результатов поиска

Из-за экономии времени* (дедлайн приближается!:с) не будем использовать "супергибкую" ссылку Super_Link.
Заменим ее на следующий список ссылок:

In [80]:
Okruga = [\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=23&district%5B1%5D=24&district%5B10%5D=33&district%5B11%5D=34&district%5B12%5D=35&district%5B13%5D=36&district%5B14%5D=37&district%5B15%5D=38&district%5B2%5D=25&district%5B3%5D=26&district%5B4%5D=27&district%5B5%5D=28&district%5B6%5D=29&district%5B7%5D=30&district%5B8%5D=31&district%5B9%5D=32&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=39&district%5B1%5D=40&district%5B10%5D=49&district%5B11%5D=50&district%5B12%5D=51&district%5B13%5D=52&district%5B14%5D=53&district%5B15%5D=54&district%5B16%5D=55&district%5B2%5D=41&district%5B3%5D=42&district%5B4%5D=43&district%5B5%5D=44&district%5B6%5D=45&district%5B7%5D=46&district%5B8%5D=47&district%5B9%5D=48&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=56&district%5B1%5D=57&district%5B10%5D=66&district%5B11%5D=67&district%5B12%5D=68&district%5B13%5D=69&district%5B14%5D=70&district%5B15%5D=71&district%5B2%5D=58&district%5B3%5D=59&district%5B4%5D=60&district%5B5%5D=61&district%5B6%5D=62&district%5B7%5D=63&district%5B8%5D=64&district%5B9%5D=65&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=72&district%5B1%5D=73&district%5B10%5D=82&district%5B11%5D=83&district%5B2%5D=74&district%5B3%5D=75&district%5B4%5D=76&district%5B5%5D=77&district%5B6%5D=78&district%5B7%5D=79&district%5B8%5D=80&district%5B9%5D=81&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=84&district%5B1%5D=85&district%5B10%5D=94&district%5B11%5D=95&district%5B12%5D=96&district%5B13%5D=97&district%5B14%5D=98&district%5B15%5D=99&district%5B2%5D=86&district%5B3%5D=87&district%5B4%5D=88&district%5B5%5D=89&district%5B6%5D=90&district%5B7%5D=91&district%5B8%5D=92&district%5B9%5D=93&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=100&district%5B1%5D=101&district%5B10%5D=110&district%5B11%5D=111&district%5B2%5D=102&district%5B3%5D=103&district%5B4%5D=104&district%5B5%5D=105&district%5B6%5D=106&district%5B7%5D=107&district%5B8%5D=108&district%5B9%5D=109&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=112&district%5B1%5D=113&district%5B10%5D=122&district%5B11%5D=123&district%5B12%5D=124&district%5B13%5D=348&district%5B14%5D=349&district%5B15%5D=350&district%5B2%5D=114&district%5B3%5D=115&district%5B4%5D=116&district%5B5%5D=117&district%5B6%5D=118&district%5B7%5D=119&district%5B8%5D=120&district%5B9%5D=121&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',\
'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=125&district%5B1%5D=126&district%5B2%5D=127&district%5B3%5D=128&district%5B4%5D=129&district%5B5%5D=130&district%5B6%5D=131&district%5B7%5D=132&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'\
            ]
# Каждая ссылка из списка - это ссылка на квартиры из одного округа* с возможностью гулять по страницам.
# 9 ссылок <=> рассматриваю 9 округов.

*В теории данная замена должна была быть эквивалентной...Но из-за самого сайта ЦИАН ссылка на округ на самом деле выдает не все квартиры в данном округе - т.е. меньше, чем если бы мы воспользовались Super_Link и прошлись по всем районам (districts), соответствующим этому округу => Экономим время, но выкачиваем не все квартиры (опять же, если это нас не устраивает - можно воспользоваться Super_Link).

Создадим DataFrame, в который в дальнейшем (после выполнения цикла) поместим информацию по квартирам (параметры).

In [81]:
Data_Flats = pd.DataFrame(columns=['n','rooms','price','totsp','livesp','kitsp','dist','metrdist','walk','brick','tel','bal','floor','nfloors','new'])

# (цикл:)

Цикл должен пройти по всем округам и по всем страницам внутри каждого округа.
Я проверил, что максимальное количество страниц для округа - 30.

=> План действий: Из каждого округа выкачиваем 30 страниц -> используем уже определенную "Вспомогательную функцию №2": delete_dublicates(.) для удаления повторяющихся квартир (повторения возникают из-за самого сайта) -> применяем уже определенные "Функции для извлечения параметров" на полученном массиве -> результаты записываем в соответствующие параметры -> а параметры в Data_Flats

In [82]:
#Заметим, что ссылка на любую квартиру имеет следующий вид:
# http://www.cian.ru/sale/flat/*********, где * - цифра.
#Поэтому, давайте заранее определим следующее выражение, в которое потом будем подставлять нужные цифры:
links_basis = "http://www.cian.ru/sale/flat/{}" #(в {} для каждой квартиры будем подставлять свою "численную часть", которую я назвал: "link") 

In [85]:
n_count = 0
for okrug in Okruga:
    
    links = []
    for page in range(1, 31):
        try:
            page_url =  okrug.format(page)

            search_page = requests.get(page_url)
            search_page = search_page.content
            search_page = BeautifulSoup(search_page, 'lxml')

            flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
            flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))


            for link in flat_urls:
                if link.isdigit():
                    links.append(link)
        except:
            print(':с')
            
    links = delete_dublicates(links)
    
    for link in links:
        try:
            n_count = n_count + 1

            flat_page = requests.get(links_basis.format(link))
            flat_page = flat_page.content
            flat_page = BeautifulSoup(flat_page, 'lxml')
            appendix = {'n':n_count,\
                        'rooms':getRoom(flat_page),\
                        'price':getPrice(flat_page),\
                        'totsp':getTotsp(flat_page),\
                        'livesp':getLivesp(flat_page),\
                        'kitsp':getKitsp(flat_page),\
                        'dist':getDist(flat_page),\
                        'metrdist':getMetrdist(flat_page),\
                        'walk':getWalk(flat_page),\
                        'brick':getBrick(flat_page),\
                        'tel':getTel(flat_page),\
                        'bal':getBal(flat_page),\
                        'floor':getFloor(flat_page),\
                        'nfloors':getNfloors(flat_page),\
                        'new':getNew(flat_page),\
                        }
            Data_Flats = Data_Flats.append(appendix, ignore_index=True) #Это итоговый массив с квартирами и их параметрами.
        except:
            print(':C')

In [86]:
Data_Flats.to_csv('Data_Flats.csv')